In [ ]:
import datetime
HEADER="""DELETE FROM public."User"; INSERT INTO public."User"(
	"Id", "Name", "BirthDate", "Gender", "ValidFrom", "ValidTo", "LocationId", "SegmentId") VALUES
"""
ROW="( {user_id}, '{name}', '{birth_date}','{gender}', '{from_date}', '{to_date}', {location_id}, {segment_id} )"
RECORDS=1000
LOCATION_RANGE=(1,17) # 17 locations
SEGMENT_RANGE=(1,3) # 3 segments
FUTURE=datetime.date(2999,12,31)
GENDERS=["M","F","O"]
BIRTH_DATE_START=datetime.date(1950,1,1)
BIRTH_DATE_RANGE=365*55 # 55 years
SEGMENT_AGES_RANGES=[(0,25),(25,45),(45,100)] # 3 segments
INSERT_USERS_FILE="../15_insert_users.sql"

In [ ]:
import faker
import random

faker_name = faker.Faker()
today = datetime.date.today()
insert_users = []
for i in range(1,RECORDS+1):
    name = faker_name.name()
    
    birth_date = BIRTH_DATE_START + datetime.timedelta(days=random.randint(0,BIRTH_DATE_RANGE))
    from_date = today - datetime.timedelta(days=random.randint(180,365*5))
    location_id = random.randint(*LOCATION_RANGE)
    gender = random.choice(GENDERS)
    age = (today - birth_date).days // 365
    age_segment = list(filter(lambda x : x[0] <= age < x[1], SEGMENT_AGES_RANGES))
    segment_id = random.randint(*SEGMENT_RANGE) + SEGMENT_AGES_RANGES.index(age_segment[0]) * 3
    insert_users.append(
        dict(
            user_id=i,
            name= name,
            birth_date= birth_date,
            from_date= from_date,
            to_date= FUTURE,
            gender=gender,
            location_id=location_id,
            segment_id=segment_id
        )
    )
    print(ROW.format(**insert_users[-1]))



In [ ]:
## MAKE INSERT STATEMENT
insert_sql = f'{HEADER}'
insert_sql += ',\n'.join([ROW.format(**user) for user in insert_users])
insert_sql += ';'
print(insert_sql)


In [ ]:
with open(INSERT_USERS_FILE, 'w') as f:
    f.write(insert_sql)

In [ ]:
#### Set user subscriptions
HEADER="""DELETE FROM public."UserSubscription"; INSERT INTO public."UserSubscription"(
    "UserId", "SubscriptionId", "ValidFrom", "ValidTo") VALUES
"""
ROW="( {user_id}, {subscription_id}, '{from_date}', '{to_date}' )"
SUBSCRIPTIONS_RANGE=(1,3) # 3 subscriptions
CHANGE_SUB_ONE_TIME=0.5
CHANGE_SUB_TWO_TIMES=0.3

In [ ]:
# Set initial subs for every user, sharing same start date
insert_subscriptions = []
for user in insert_users:
    insert_subscriptions.append(
        dict(
            user_id=user['user_id'],
            subscription_id=random.randint(*SUBSCRIPTIONS_RANGE),
            from_date=user['from_date'],
            to_date=user['from_date'] + datetime.timedelta(days=round((datetime.date.today() - user['from_date']).days * random.random()))
        )
    )
    print(ROW.format(**insert_subscriptions[-1]))

In [ ]:
lost_customers_index = random.sample(
    range(0,len(insert_subscriptions)), 
    int((CHANGE_SUB_ONE_TIME+CHANGE_SUB_TWO_TIMES)*len(insert_subscriptions))
)
len(lost_customers_index)


In [ ]:
# Change subscription once
from collections import defaultdict


cut_index = int(CHANGE_SUB_ONE_TIME*len(insert_users))
for i in lost_customers_index:
    new_sub = insert_subscriptions[i]['subscription_id']
    while insert_subscriptions[i]['subscription_id'] == (new_sub := random.randint(*SUBSCRIPTIONS_RANGE)): ... 
    insert_subscriptions.append(
        dict(
            user_id=insert_subscriptions[i]['user_id'],
            subscription_id=new_sub ,
            from_date=insert_subscriptions[i]['to_date'] + datetime.timedelta(days=1),
            to_date=FUTURE
        )
    )
# Change subscription twice    
for i in lost_customers_index[cut_index:]:
    # Find a new subscription change date
    days_first_sub = int((today - insert_subscriptions[i]['to_date']).days * random.random())
    # Change to date from previous subscription
    insert_subscriptions[i]['to_date'] = insert_subscriptions[i]['from_date'] + datetime.timedelta(days=days_first_sub)
    # Add new subscription
    while insert_subscriptions[i]['subscription_id'] == (new_sub := random.randint(*SUBSCRIPTIONS_RANGE)): ... 
    insert_subscriptions.append(
        dict(
            user_id=insert_subscriptions[i]['user_id'],
            subscription_id=new_sub ,
            from_date=insert_subscriptions[i]['to_date'] + datetime.timedelta(days=1),
            to_date=FUTURE
        )
    )

  

In [ ]:
# Check users with miltiple rows
repeated_user_id_count = defaultdict(int)
for item in insert_subscriptions:
    print(repeated_user_id_count[item['user_id']])
    repeated_user_id_count[item['user_id']] += 1
count = [0,0,0]
for v in repeated_user_id_count.values():
    count[v-1] += 1
print(f'Users with 1 subscription: {count[0]}')
print(f'Users with 2 subscriptions: {count[1]}')
print(f'Users with 3 subscriptions: {count[2]}')  

In [ ]:
len(insert_subscriptions)

In [ ]:
## Create user subscriptions insert statement
insert_sql = f'{HEADER}'
insert_sql += ',\n'.join([ROW.format(**sub) for sub in insert_subscriptions])
insert_sql += ';'   

print(insert_sql)

In [ ]:
# Write to file
INSERT_SUBSCRIPTIONS_FILE="../16_insert_user_subscriptions.sql"
with open(INSERT_SUBSCRIPTIONS_FILE, 'w') as f:
    f.write(insert_sql)